<a href="https://colab.research.google.com/github/trippyvortex/Clarity_Colab_Notebook/blob/main/ClarityUpscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install replicate

In [ ]:
import os

# Set the environment variable
os.environ['REPLICATE_API_TOKEN'] = 'repalce this text with your replicate api token'

# Verify that the environment variable is set
print(os.environ['REPLICATE_API_TOKEN'])


In [ ]:
import replicate

In [ ]:
output = replicate.run(
    "philz1337x/clarity-upscaler:f11a4727f8f995d2795079196ebda1bcbc641938e032154f46488fc3e760eb79",
    input={
        "seed": 1337,
        "image": "/content/frame0001.jpg",
        "prompt": "masterpiece, best quality, highres, <lora:more_details:0.5> <lora:SDXLrender_v2.0:1>",
        "dynamic": 6,
        "handfix": "disabled",
        "sharpen": 0,
        "sd_model": "juggernaut_reborn.safetensors [338b85bc4f]",
        "scheduler": "DPM++ 3M SDE Karras",
        "creativity": 0.35,
        "lora_links": "",
        "downscaling": False,
        "resemblance": 0.6,
        "scale_factor": 2,
        "tiling_width": 112,
        "output_format": "png",
        "tiling_height": 144,
        "custom_sd_model": "",
        "negative_prompt": "(worst quality, low quality, normal quality:2) JuggernautNegative-neg",
        "num_inference_steps": 18,
        "downscaling_resolution": 768
    }
)
print(output)

#Try this below for uploading a short video. I couldnt get it to work before running out of replicate free time, but it may work.

In [ ]:
# Install necessary libraries
!pip install opencv-python-headless
!pip install ffmpeg-python
!pip install tqdm
!pip install replicate
from google.colab import files

uploaded = files.upload()



In [ ]:
import os
import cv2
import ffmpeg

uploaded_file = list(uploaded.keys())[0]
input_path = f'/content/{uploaded_file}'

if uploaded_file.endswith(('.mp4', '.avi', '.mov')):
    # Create a directory to save frames
    os.makedirs('frames', exist_ok=True)

    # Capture the video
    cap = cv2.VideoCapture(input_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(f'frames/frame_{count:04d}.png', frame)
        count += 1

    cap.release()
    cv2.destroyAllWindows()
    print(f'Extracted {count} frames from the video.')
else:
    # Assuming uploaded files are images in a folder
    os.makedirs('frames', exist_ok=True)
    for file_name in uploaded.keys():
        os.rename(f'/content/{file_name}', f'frames/{file_name}')
    frame_count = len(uploaded)
    frame_rate = 24  # Default FPS for images, can be changed based on user input
    print(f'Uploaded {frame_count} images.')


In [ ]:
import replicate
from tqdm import tqdm
import requests

upscaled_dir = 'upscaled_frames'
os.makedirs(upscaled_dir, exist_ok=True)

# Replace the API token with your actual token
REPLICATE_API_TOKEN = 'replace this text with your replicate api token'
replicate.Client(api_token=REPLICATE_API_TOKEN)

for frame_file in tqdm(sorted(os.listdir('frames'))):
    frame_path = os.path.join('frames', frame_file)
    output = replicate.run(
        "philz1337x/clarity-upscaler:f11a4727f8f995d2795079196ebda1bcbc641938e032154f46488fc3e760eb79",
        input={
            "seed": 1337,
            "image": open(frame_path, "rb"),
            "prompt": "masterpiece, best quality, highres, <lora:more_details:0.5> <lora:SDXLrender_v2.0:1>",
            "dynamic": 6,
            "handfix": "disabled",
            "sharpen": 0,
            "sd_model": "juggernaut_reborn.safetensors [338b85bc4f]",
            "scheduler": "DPM++ 3M SDE Karras",
            "creativity": 0.35,
            "lora_links": "",
            "downscaling": False,
            "resemblance": 0.6,
            "scale_factor": 2,
            "tiling_width": 112,
            "output_format": "png",
            "tiling_height": 144,
            "custom_sd_model": "",
            "negative_prompt": "(worst quality, low quality, normal quality:2) JuggernautNegative-neg",
            "num_inference_steps": 18,
            "downscaling_resolution": 768
        }
    )

    # Output from replicate.run is a list of URLs
    upscaled_image_url = output[0]

    # Download the upscaled image
    response = requests.get(upscaled_image_url)
    upscaled_frame_path = os.path.join(upscaled_dir, frame_file)

    with open(upscaled_frame_path, 'wb') as f:
        f.write(response.content)
